In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from sklearn.cluster import DBSCAN

In [2]:
import os
if not os.path.exists("./data"):
    os.makedirs("./data")
if not os.path.exists("./data/images"):
    os.makedirs("./data/images")
if not os.path.exists("./data/labels"):
    os.makedirs("./data/labels")

In [3]:
satimg = rasterio.open('D:/JOB/Pruebas/SatelliteImage_ObjectDetection/sat_img/SAS_satimg_google20_02.tif')
samples = gpd.read_file('D:/JOB/Pruebas/SatelliteImage_ObjectDetection/sample_layer/sat_img_sample_class.shp')
print(len(samples))

img_h = satimg.shape[0]
img_w = satimg.shape[1]
print(f"Altura: {img_h}")
print(f"Ancho: {img_w}")

img_long = list(satimg.bounds)[2] - list(satimg.bounds)[0]
img_lat = list(satimg.bounds)[3] - list(satimg.bounds)[1]
print('long :: ', img_long, satimg.shape[1])
print('lat  :: ', img_lat, satimg.shape[0])

samples_bounds = samples.bounds
samples = pd.concat([samples, samples_bounds], axis=1)
samples.head()

63
Altura: 8941
Ancho: 11489
long ::  0.03081589937210083 11489
lat  ::  0.02398163080215454 8941


,id,class,geometry,minx,miny,maxx,maxy
0,1,pool,"POLYGON ((-60.98276 -34.62037, -60.98267 -34.6...",-60.982759,-34.620425,-60.982602,-34.620298
1,2,pool,"POLYGON ((-60.98323 -34.62100, -60.98323 -34.6...",-60.983233,-34.621019,-60.983092,-34.620933
2,3,pool,"POLYGON ((-60.98317 -34.62076, -60.98312 -34.6...",-60.983169,-34.620816,-60.983028,-34.620686
3,4,pool,"POLYGON ((-60.98241 -34.61976, -60.98232 -34.6...",-60.982405,-34.619797,-60.982262,-34.619683
4,5,pool,"POLYGON ((-60.98268 -34.61958, -60.98261 -34.6...",-60.982684,-34.619621,-60.982560,-34.619516


In [4]:
samples['x_cen'] = samples.apply(lambda row: row.geometry.centroid.coords[0][0], axis=1)
samples['y_cen'] = samples.apply(lambda row: row.geometry.centroid.coords[0][1], axis=1)
samples['x_cen_px'] = samples.apply(lambda row: round(((row.x_cen- list(satimg.bounds)[0]) * img_w) / img_long), axis=1)
samples['x_min_px'] = samples.apply(lambda row: round(((row.minx- list(satimg.bounds)[0]) * img_w) / img_long), axis=1)
samples['x_max_px'] = samples.apply(lambda row: round(((row.maxx- list(satimg.bounds)[0]) * img_w) / img_long), axis=1)
samples['y_cen_px'] = samples.apply(lambda row: img_h - round(((row.y_cen- list(satimg.bounds)[1]) * img_h) / img_lat), axis=1)
samples['y_min_px'] = samples.apply(lambda row: img_h - round(((row.miny- list(satimg.bounds)[1]) * img_h) / img_lat), axis=1)
samples['y_max_px'] = samples.apply(lambda row: img_h - round(((row.maxy- list(satimg.bounds)[1]) * img_h) / img_lat), axis=1)
samples['bbox_bound'] = samples.apply(lambda row: f"[[{row.x_min_px}, {row.y_min_px}], [{row.x_max_px}, {row.y_max_px}]]", axis=1)
samples.head(3)

,id,class,geometry,minx,miny,maxx,maxy,x_cen,y_cen,x_cen_px,x_min_px,x_max_px,y_cen_px,y_min_px,y_max_px,bbox_bound
0,1,pool,"POLYGON ((-60.98276 -34.62037, -60.98267 -34.6...",-60.982759,-34.620425,-60.982602,-34.620298,-60.982682,-34.620361,3177,3148,3206,3807,3830,3783,"[[3148, 3830], [3206, 3783]]"
1,2,pool,"POLYGON ((-60.98323 -34.62100, -60.98323 -34.6...",-60.983233,-34.621019,-60.983092,-34.620933,-60.983163,-34.620976,2998,2971,3024,4036,4052,4020,"[[2971, 4052], [3024, 4020]]"
2,3,pool,"POLYGON ((-60.98317 -34.62076, -60.98312 -34.6...",-60.983169,-34.620816,-60.983028,-34.620686,-60.983098,-34.620752,3022,2995,3048,3952,3976,3928,"[[2995, 3976], [3048, 3928]]"


In [5]:
coords, center_list = [], []
for p in range(len(samples)):
    coords.append([[samples.loc[p, 'x_min_px'], samples.loc[p, 'y_min_px']], 
                   [samples.loc[p, 'x_max_px'], samples.loc[p, 'y_max_px']]])
    
    # stores box's center coords for clustering
    center_list.append([samples.loc[p, 'x_cen_px'], samples.loc[p, 'y_cen_px']])
coords = np.array(coords)

In [9]:
# DB-Scan algorithm for clustering
# value of eps (threshold) can be set from 220-256
# depending on the number of clusters
eps = round(512 * 0.40)
print(eps)
dbscan = DBSCAN(min_samples=1, eps=eps)
x = np.array(center_list)
y = dbscan.fit_predict(x)

# storing centroid of clusters
info = {}
for i in range(y.max()+1):
    mi_x, mi_y = x[np.where(y==i)[0]].min(axis=0)
    ma_x, ma_y = x[np.where(y==i)[0]].max(axis=0)
    print(f"{i}: {mi_x} - {ma_x} / {mi_y} - {ma_y} :: center = {(mi_x+ma_x)//2} - {(mi_y+ma_y)//2}")

    item = {}
    item['center'] = [(mi_x+ma_x)//2, (mi_y+ma_y)//2]
    item['bbox'] = coords[np.where(y==i)[0]].tolist()

    info[i] = item

#*****************************************************************************************
df_data = pd.DataFrame(info.items(), columns=['id', 'dict_data'])
df_data['bboxes'] = df_data.apply(lambda row: row['dict_data']['bbox'], axis=1)

list_df_bboxes = []
for id in list(df_data.id):
    df_bboxes = pd.DataFrame(df_data.loc[id, 'bboxes'], columns=['b_bot', 'b_top'])
    df_bboxes['class_dbscan'] = df_data.loc[id, 'id']
    list_df_bboxes.append(df_bboxes)

df_bboxes = pd.concat(list_df_bboxes)
df_bboxes['bbox_xmin'] = df_bboxes.apply(lambda row: row.b_bot[0], axis=1)
df_bboxes['bbox_ymin'] = df_bboxes.apply(lambda row: row.b_bot[1], axis=1)
df_bboxes['bbox_bound'] = df_bboxes.apply(lambda row: f"[{row.b_bot}, {row.b_top}]", axis=1)

samples_bbox = samples.merge(df_bboxes, left_on='bbox_bound', right_on='bbox_bound') # [samples.x_min_px == df_bboxes.bbox_xmin]
samples_bbox[['id', 'class', 'class_dbscan', 'geometry']].to_file('./data/samples_classified.geojson', driver='GeoJSON')
print(f"{len(samples_bbox)} / {len(samples)}")

205
0: 3177 - 3177 / 3807 - 3807 :: center = 3177 - 3807
1: 2998 - 3022 / 3952 - 4036 :: center = 3010 - 3994
2: 3199 - 3453 / 3330 - 3576 :: center = 3326 - 3453
3: 3474 - 3575 / 3121 - 3182 :: center = 3524 - 3151
4: 3771 - 4013 / 3374 - 3558 :: center = 3892 - 3466
5: 4268 - 4268 / 3581 - 3581 :: center = 4268 - 3581
6: 4343 - 4456 / 3738 - 3835 :: center = 4399 - 3786
7: 4471 - 4933 / 3876 - 4205 :: center = 4702 - 4040
8: 4179 - 4442 / 4298 - 4488 :: center = 4310 - 4393
9: 4065 - 4065 / 4027 - 4027 :: center = 4065 - 4027
10: 3860 - 3867 / 3881 - 3994 :: center = 3863 - 3937
11: 3252 - 3575 / 4095 - 4384 :: center = 3413 - 4239
12: 2753 - 3395 / 4288 - 4788 :: center = 3074 - 4538
13: 3775 - 4167 / 4501 - 4801 :: center = 3971 - 4651
14: 3742 - 3742 / 4974 - 4974 :: center = 3742 - 4974
15: 3387 - 3597 / 4806 - 4967 :: center = 3492 - 4886
16: 3559 - 3559 / 5132 - 5132 :: center = 3559 - 5132
17: 2796 - 2796 / 3825 - 3825 :: center = 2796 - 3825
63 / 63


In [10]:
img_chip_size = 512
points_delete = []
samples_bbox_clean = samples_bbox.copy()

for k in info.keys():
    x, y = info[k]['center'][0], info[k]['center'][1]

    # saving image chip

    window = Window(x-img_chip_size/2, y-img_chip_size/2, img_chip_size, img_chip_size)
    satimg_transform = satimg.window_transform(window)
    # Create a new cropped raster to write to
    profile = satimg.profile
    profile.update({
        'height': img_chip_size,
        'width': img_chip_size,
        'transform': satimg_transform})
    print(f"output_{k}.tif")
    with rasterio.open(f"./data/images/output_{k}.tif", 'w', **profile) as dst:
        # Read the data from the window and write it to the output raster
        dst.write(satimg.read(window=window))
    
    # saving label
    file = open(f"./data/labels/output_{k}.txt", 'a')
    for point in info[k]['bbox']:
        [[x_bot, y_bot], [x_top, y_top]] = point

        if x-img_chip_size/2 < ((x_bot+x_top)//2) < x+img_chip_size/2:
            xc = (x_bot+x_top)//2 - x + img_chip_size/2
            yc = (y_bot+y_top)//2 - y + img_chip_size/2
            w = abs(x_bot-x_top)
            h = abs(y_bot-y_top)

            # 0 means first object i.e. ship
            lab = '0 {} {} {} {}\n'.format(xc/img_chip_size, yc/img_chip_size, w/img_chip_size, h/img_chip_size)
            file.write(lab)
        else:
            delete_bbox_point = f"[[{x_bot}, {y_bot}], [{x_top}, {y_top}]]"
            points_delete.append(delete_bbox_point)
            samples_bbox_clean = samples_bbox_clean.loc[samples_bbox_clean.bbox_bound != delete_bbox_point]
    file.close()

samples_bbox_clean[['id', 'class', 'class_dbscan', 'geometry']].to_file('./data/samples_classified_clean.geojson', driver='GeoJSON')

output_0.tif
output_1.tif
output_2.tif
output_3.tif
output_4.tif
output_5.tif
output_6.tif
output_7.tif
output_8.tif
output_9.tif
output_10.tif
output_11.tif
output_12.tif
output_13.tif
output_14.tif
output_15.tif
output_16.tif
output_17.tif
